In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

class TimeSeriesPredictor:
    def __init__(self, window_size=60):
        self.window_size = window_size
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None
        
    def create_model(self):
        """Создать LSTM модель с указанной архитектурой: LSTM(50) → Dense(25) → Dense(1)"""
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=False, input_shape=(self.window_size, 1)))
        model.add(Dense(units=25, activation='relu'))
        model.add(Dense(units=1))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
        return model
    
    def prepare_data(self, data):
        """Создать скользящие окна из данных"""
        X, y = [], []
        for i in range(self.window_size, len(data)):
            X.append(data[i-self.window_size:i, 0])
            y.append(data[i, 0])
        return np.array(X), np.array(y)
    
    def normalize(self, data):
        """Нормализовать данные (Min-Max scaling)"""
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        normalized_data = self.scaler.fit_transform(data)
        return normalized_data
    
    def denormalize(self, data):
        """Денормализовать данные"""
        denormalized_data = self.scaler.inverse_transform(data)
        return denormalized_data
    
    def train(self, data, epochs=50, batch_size=32, validation_split=0.1):
        """Обучить модель"""
        # Нормализация данных
        normalized_data = self.normalize(data)
        
        # Подготовка данных
        X, y = self.prepare_data(normalized_data)
        
        # Изменение формы для LSTM [samples, time steps, features]
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))
        
        # Разделение на обучающую и тестовую выборки (80/20)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]
        
        # Создание модели
        self.model = self.create_model()
        
        # Обучение модели
        print("Обучение модели...")
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            verbose=1
        )
        
        return history, X_test, y_test
    
    def predict(self, last_window):
        """Предсказать следующее значение"""
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод train().")
        
        # Подготовка данных для предсказания
        last_window_normalized = self.scaler.transform(last_window.reshape(-1, 1))
        X_pred = np.array([last_window_normalized[-self.window_size:, 0]])
        X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))
        
        # Предсказание
        prediction_normalized = self.model.predict(X_pred)
        prediction = self.denormalize(prediction_normalized)
        
        return prediction[0][0]
    
    def evaluate(self, X_test, y_test):
        """Оценить модель на тестовых данных"""
        # Предсказания на тестовых данных
        y_pred_normalized = self.model.predict(X_test)
        y_pred = self.denormalize(y_pred_normalized)
        y_test_original = self.denormalize(y_test.reshape(-1, 1))
        
        # Расчет метрик
        mse = np.mean((y_pred - y_test_original) ** 2)
        mae = np.mean(np.abs(y_pred - y_test_original))
        rmse = np.sqrt(mse)
        
        return y_pred, y_test_original, mse, mae, rmse

def generate_synthetic_stock_data(num_points=1000):
    """Генерация синтетических данных цен акций"""
    np.random.seed(42)
    time = np.arange(0, num_points)
    
    # Тренд
    trend = time * 0.1
    
    # Сезонность
    seasonality = 50 * np.sin(time * 0.05)
    
    # Шум
    noise = np.random.normal(0, 10, num_points)
    
    # Объединение компонентов
    stock_prices = 100 + trend + seasonality + noise
    
    # Добавление некоторых резких изменений (имитация рыночных событий)
    changes = np.random.choice([-1, 0, 1], num_points, p=[0.05, 0.9, 0.05])
    abrupt_changes = np.cumsum(changes * 20)
    
    final_prices = stock_prices + abrupt_changes
    return final_prices.reshape(-1, 1)

def visualize_results(history, y_test, y_pred, original_data, window_size):
    """Визуализация результатов обучения и предсказаний"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. График потерь при обучении
    axes[0, 0].plot(history.history['loss'], label='Обучающая выборка')
    axes[0, 0].plot(history.history['val_loss'], label='Валидационная выборка')
    axes[0, 0].set_title('Функция потерь при обучении')
    axes[0, 0].set_xlabel('Эпоха')
    axes[0, 0].set_ylabel('Потери (MSE)')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # 2. Сравнение реальных и предсказанных значений
    test_indices = np.arange(len(y_test))
    axes[0, 1].plot(test_indices, y_test, label='Реальные значения', alpha=0.7)
    axes[0, 1].plot(test_indices, y_pred, label='Предсказания', alpha=0.7)
    axes[0, 1].set_title('Сравнение реальных и предсказанных значений')
    axes[0, 1].set_xlabel('Индекс тестовой выборки')
    axes[0, 1].set_ylabel('Цена')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # 3. Полный ряд данных с выделением тестовой части
    train_size = int(0.8 * (len(original_data) - window_size))
    train_data = original_data[:train_size + window_size]
    test_data = original_data[train_size + window_size:]
    
    axes[1, 0].plot(np.arange(len(train_data)), train_data, label='Обучающие данные', alpha=0.7)
    axes[1, 0].plot(np.arange(len(train_data), len(original_data)), test_data, label='Тестовые данные', alpha=0.7)
    axes[1, 0].plot(np.arange(len(train_data), len(original_data)), y_pred, label='Предсказания', alpha=0.9, linestyle='--')
    axes[1, 0].set_title('Полный временной ряд с предсказаниями')
    axes[1, 0].set_xlabel('Время')
    axes[1, 0].set_ylabel('Цена')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # 4. Ошибки предсказаний
    errors = y_test - y_pred
    axes[1, 1].plot(test_indices, errors, label='Ошибки предсказаний', color='red', alpha=0.7)
    axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1, 1].fill_between(test_indices, errors.flatten(), 0, alpha=0.3, color='red')
    axes[1, 1].set_title('Ошибки предсказаний')
    axes[1, 1].set_xlabel('Индекс тестовой выборки')
    axes[1, 1].set_ylabel('Ошибка')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

def main():
    # Генерация синтетических данных
    print("Генерация синтетических данных цен акций...")
    stock_data = generate_synthetic_stock_data(1000)
    print(f"Сгенерировано {len(stock_data)} точек данных")
    
    # Создание и обучение модели
    predictor = TimeSeriesPredictor(window_size=60)
    
    # Обучение модели
    history, X_test, y_test = predictor.train(stock_data, epochs=30, batch_size=16)
    
    # Оценка модели
    y_pred, y_test_original, mse, mae, rmse = predictor.evaluate(X_test, y_test)
    
    # Вывод метрик
    print("\n" + "="*50)
    print("МЕТРИКИ КАЧЕСТВА МОДЕЛИ:")
    print("="*50)
    print(f"Среднеквадратичная ошибка (MSE): {mse:.4f}")
    print(f"Средняя абсолютная ошибка (MAE): {mae:.4f}")
    print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.4f}")
    print(f"Среднее значение цены: {np.mean(stock_data):.2f}")
    print(f"Относительная ошибка (RMSE/Среднее): {(rmse/np.mean(stock_data)*100):.2f}%")
    
    # Демонстрация предсказания следующего значения
    print("\n" + "="*50)
    print("ПРЕДСКАЗАНИЕ СЛЕДУЮЩЕГО ЗНАЧЕНИЯ:")
    print("="*50)
    
    # Берем последнее окно из тестовых данных
    last_window = stock_data[-(predictor.window_size + 10):-10]
    next_value_prediction = predictor.predict(last_window)
    
    # Фактическое следующее значение (для сравнения)
    actual_next_value = stock_data[-10]
    
    print(f"Предсказанное следующее значение: {next_value_prediction:.2f}")
    print(f"Фактическое следующее значение: {actual_next_value[0]:.2f}")
    print(f"Ошибка предсказания: {abs(next_value_prediction - actual_next_value[0]):.2f}")
    
    # Визуализация результатов
    visualize_results(history, y_test_original, y_pred, stock_data, predictor.window_size)
    
    # Структура модели
    print("\n" + "="*50)
    print("СТРУКТУРА МОДЕЛИ:")
    print("="*50)
    predictor.model.summary()

if __name__ == "__main__":
    main()